In [1]:
import pandas as pd
import numpy as np
import pickle

from random import choice

# from crossref.restful import Works, Etiquette
# from scihub_blastoise import SciHub_HydroPump


### PART 1: Partial Run SumCetCet Analysis

We're back after Barça!

- The first order of business is to combine all partial fill SumCetCet run data to a single dataset to analyze.


In [7]:
# Sum:

with open("sum_pr_cr_results", "rb") as fp:
    sum_pr_cr_results = pickle.load(fp)

with open("sum_pr_sh_succ", "rb") as fp:
    sum_pr_sh_succ = pickle.load(fp)

with open("sum_pr_sh_fail", "rb") as fp:
    sum_pr_sh_fail = pickle.load(fp)

In [3]:
# Cet

with open("cet_pr_cr_results", "rb") as fp:
    cet_pr_cr_results = pickle.load(fp)

with open("cet_pr_sh_succ", "rb") as fp:
    cet_pr_sh_succ = pickle.load(fp)

with open("cet_pr_sh_fail", "rb") as fp:
    cet_pr_sh_fail = pickle.load(fp)

In [4]:
# Ogi:

with open("pr_cr_results", "rb") as fp:
    ogi_pr_cr_results = pickle.load(fp)

with open("pr_sh_succ", "rb") as fp:
    ogi_pr_sh_succ = pickle.load(fp)

with open("pr_sh_fail", "rb") as fp:
    ogi_pr_sh_fail = pickle.load(fp)

The initial analysis shows that around 50% of the metadata retr. from CR (pr_cr_results), was used for SH date extraction phase.

However, 40-50% of these articles actually contaion DATE info, therefore although the Req_Count is more than satisfied in the CR phase, SH results still do not satisfy for all journals.

Once the analysis is complete, we can use the pr_cr_results, to filter out the UNTESTED artcls, and pass them through the SH process.

In [46]:
sum_pr_cr_results = pd.DataFrame(sum_pr_cr_results)
sum_pr_sh_succ = pd.DataFrame(sum_pr_sh_succ)
sum_pr_sh_fail = pd.DataFrame(sum_pr_sh_fail)

sum_sh_results =  pd.concat([sum_pr_sh_succ, sum_pr_sh_fail])

sum_pr_results = pd.merge(sum_pr_cr_results, sum_sh_results, how="left", on="DOI")


In [47]:
cet_pr_cr_results = pd.DataFrame(cet_pr_cr_results)
cet_pr_sh_succ = pd.DataFrame(cet_pr_sh_succ)
cet_pr_sh_fail = pd.DataFrame(cet_pr_sh_fail)

cet_sh_results =  pd.concat([cet_pr_sh_succ, cet_pr_sh_fail])

cet_pr_results = pd.merge(cet_pr_cr_results, cet_sh_results, how="left", on="DOI")


In [48]:
ogi_pr_cr_results = pd.DataFrame(ogi_pr_cr_results)
ogi_pr_sh_succ = pd.DataFrame(ogi_pr_sh_succ)
ogi_pr_sh_fail = pd.DataFrame(ogi_pr_sh_fail)

ogi_sh_results =  pd.concat([ogi_pr_sh_succ, ogi_pr_sh_fail])

ogi_pr_results = pd.merge(ogi_pr_cr_results, ogi_sh_results, how="left", on="DOI")

Concatinating all partial run results to a single dataset: (pr_results_df)

In [54]:
pr_results_df = pd.concat([sum_pr_results, cet_pr_results,ogi_pr_results], ignore_index=True, verify_integrity=True)

In [59]:
pr_results_df.Dates.map(type).value_counts()

<class 'float'>    41840
<class 'str'>      35472
<class 'list'>     24708
Name: Dates, dtype: int64

Now that all the partial run results are consolidated:

TTD:
- Need to filter out the succesfull artcls, pass them through pre-processing, add them to q1_date_df

- Need to filter out FAILED, UNTESTED dfs


### PART 2: UPDATING q1_date_df

In [60]:
# TTD 1:
# Taken from 7c:

# 1.3 - Filtering to only successful artcls:
q1_date_pr = pr_results_df[pr_results_df.Dates.map(type)==list]
q1_date_pr.set_index("DOI", drop=True, inplace=True)

# 1.4 - Feature Eng. : 
def earlier_date_part(row):
    try:
        pp_year = row["published-print"]["date-parts"][0][0]
    except TypeError:
        pp_year = None

    try:   
        po_year = row["published-online"]["date-parts"][0][0]
    except TypeError:
        po_year = None

    if po_year is None:
        return row["published-print"]

    elif pp_year is None:
        return row["published-online"]

    else:
        if pp_year<po_year:
            return row["published-print"]
        elif po_year<pp_year:
            return row["published-online"]
        else:
            try:
                pp_month = row["published-print"]["date-parts"][0][1]
            except IndexError:
                pp_month = 12
            try:
                po_month = row["published-online"]["date-parts"][0][1]
            except IndexError:
                po_month = 12
            if pp_month<po_month:
                return row["published-print"]
            elif po_month<pp_month:
                return row["published-online"]
            else:
                return row["published-online"]


q1_date_pr.loc[:,"published"] = q1_date_pr.apply(earlier_date_part, axis=1)

q1_date_pr = q1_date_pr[["JRNL_ID", "ISSN", "issued", "container-title", "publisher", "title", "author", "subject", "references-count", "is-referenced-by-count", "published", "published-print", "published-online", "direct_url", "Dates", "Keywords"]]


<ipython-input-60-6f3939fe8a91>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1_date_pr.loc[:,"published"] = q1_date_pr.apply(earlier_date_part, axis=1)


__q1_date_pr__ is the succesful articles from partial run dataset:

However, it should be noted that some (also apparent in q1_date_df) only have "Accepted Date" and not "Received Date"

This MUST be dealt with later in this notebook!!!


In [ ]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

In [64]:
len(q1_date_df)

140993

In [65]:
q1_date_df = pd.concat([q1_date_df, q1_date_pr], verify_integrity=True)

__Fixing "only Accepted Date" issue:__

In [67]:
q1_date_df = q1_date_df[q1_date_df.Dates.map(lambda x: False if x[0]==None else True)]

SAVING & DUMPing all used datasets:

In [70]:
with open("q1_date_pr","wb") as p:
    pickle.dump(q1_date_pr, p)

In [121]:
# pr_results_df has ALL the extracted info from SumCetCet partial run

with open("pr_results_df","wb") as p:
    pickle.dump(pr_results_df, p)

In [69]:
with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)

Currently q1_date_df has 156754 articles that are completely ready for analysis.

The remaining TDDs are:

__1- Article related:__
- Consolidate all UNTESTED article metadata -> only available from partial fill run dataset? - DONE
- Consolidate all FAIED articles to a single dataset -> final extraction, partial fill, q1_date_df_all, OTHERS? - DONE

__2- Journal related:__
- update q1_jrnl_df and q1_jrnl_expladed with the new information
- re-calc Req_Count & check if/how many UNTESTED can be used
- compare with pivots to check total necessary data

__3- Author related:__
- Finalize the author_df 
- Complete ScopusAPI Author Search run

__4- ANALYSIS & STATS & VISUALIZE__

__5- WRITE REPORT__

__6- FIN!__


### PART 3: UNTESTED AND FAILED Article Datasets

The datasets to be considered are:

- q1_date_df_all (Backups)
- q1_meta_fe
- q1_meta_fe_all
- date_df_full (5a)
- date_df_5b (5b)
- date_df_ok (Main Files)
- date_df_ok_final (Main Files)
- pr_results_df (7e)

To prevent to many variables each will be loaded & compared with q1_date_df and saved accordingly.


In [77]:
succ_doi_list = q1_date_df.index.tolist()

In [ ]:
# Code used for testing below:

# with open("date_df_ok_final", "rb") as fp:
#     testo = pickle.load(fp)

# temp_remn = testo[~testo.set_index("DOI").index.isin(succ_doi_list)].set_index("DOI")
# temp_remn = testo[~testo.index.isin(succ_doi_list)]

# remainder_df = pd.concat([remainder_df,q1_date_pr], verify_integrity=True)
# remainder_df = pd.concat([remainder_df,temp_remn])

__Results:__

- __q1_date_df_all (Backups)__
    - Total len: 36K
    - Not in q1_date_df: 4242
    - 

- __q1_meta_fe__
    - Total len: 138,184
    - Not in q1_date_df: 35,258
    - Already included : 131
    - 

- __q1_meta_fe_all__
    - Total len: 138,184
    - Not in q1_date_df: 35,258 
    - SKIPPED AS same as the previous df
    - 
- __date_df_full (5a)__
    - Total len: 128,374
    - Not in q1_date_df: 96,156
    - Already included: 4615
    - 
- __date_df_5b (5b)__
    - SKIPPED as pnly 36,514 articles with only SciHub information
    - 
- __date_df_ok (Main Files)__
    - Total len: 114,759
    - Not in q1_date_df: 82,686 
    - Already included: 82,773 SKIPPED as a result
- __date_df_ok_final (Main Files)__
    - Total len: 114,759
    - Not in q1_date_df: 82,686 
    - SKIPPED as same as previous
    - 
- __pr_results_df (7e)__
    - Total len:102,020
    - Not in q1_date_df: 78,645 
    - Already included: 4616
    - 


All the articles are concat'd into remainder_df. This df can be used to create FAILED & UNTESTED datasets

In [129]:
with open("remainder_df","wb") as p:
    pickle.dump(remainder_df, p)

#### PART 3.1: remainder_df:

In [2]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

with open("remainder_df", "rb") as fp:
    remainder_df = pickle.load(fp)


In [5]:
remainder_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 214301 entries, 10.1007/s10441-011-9133-1 to 10.1214/18-ejp208
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ISSN                    214301 non-null  object 
 1   issued                  214301 non-null  object 
 2   container-title         214301 non-null  object 
 3   publisher               214294 non-null  object 
 4   title                   214301 non-null  object 
 5   author                  192795 non-null  object 
 6   subject                 213933 non-null  object 
 7   references-count        118145 non-null  float64
 8   is-referenced-by-count  214301 non-null  int64  
 9   published               39116 non-null   object 
 10  published-print         201171 non-null  object 
 11  published-online        119803 non-null  object 
 12  direct_url              41047 non-null   object 
 13  Dates                   41047 non-null   obj

In [8]:
# Drop unnec. colms:
remainder_df.drop(["reference","language"], axis=1, inplace=True)

In [19]:
remainder_df[remainder_df.Results.notna()].Results.map(type).value_counts()

<class 'list'>    82686
<class 'str'>     13470
Name: Results, dtype: int64

In [22]:
remainder_df[(remainder_df.Results.notna() & (remainder_df.Results.map(type)==list))]["Results"].map(len).value_counts()

2    78240
3     4446
Name: Results, dtype: int64

In [13]:
remainder_df[(remainder_df.Dates.notna()) & (remainder_df.Results.notna()) ]

,ISSN,issued,container-title,publisher,title,author,subject,references-count,is-referenced-by-count,published,published-print,published-online,direct_url,Dates,Keywords,Affiliation,JRNL_ID,reference-count,Results
DOI,,,,,,,,,,,,,,,,,,,


In [12]:
remainder_df[(remainder_df["reference-count"].notna()) & (remainder_df["references-count"].notna()) ]

,ISSN,issued,container-title,publisher,title,author,subject,references-count,is-referenced-by-count,published,published-print,published-online,direct_url,Dates,Keywords,Affiliation,JRNL_ID,reference-count,Results
DOI,,,,,,,,,,,,,,,,,,,


__remainder_df__

__1- Succ:__
- has Dates & type == list
- has Results & len == 3

__2- Fail:__
- has Dates & type==str
- has Results & len==2

__3- Untested:__
- no Results & Dates cols.
- DOI not in any other df

In [24]:
# remainder_succ:

remainder_succ = remainder_df[(remainder_df.Dates.notna() & (remainder_df.Dates.map(type)==list))]

remainder_succ = pd.concat([remainder_succ, remainder_df.fillna("-")[(remainder_df.fillna("-").Results.notna() & (remainder_df.Results.fillna("-").map(len)==3))]])

In [33]:
# remainder_fail:

remainder_fail = remainder_df[(remainder_df.Dates.notna() & (remainder_df.Dates.map(type)==str))]

remainder_fail = pd.concat([remainder_fail, remainder_df.fillna("-")[(remainder_df.fillna("-").Results.notna()) & (remainder_df.Results.fillna("-").map(len)==2)]])

In [42]:
# remainder_untested:

# Need to drop all DOIs from other datasets

date_df_dois = q1_date_df.index.to_list()

rem_suc_dois = remainder_succ.index.tolist()

rem_fail_dois = remainder_fail.index.tolist()

In [43]:
all_dois = set(date_df_dois + rem_suc_dois + rem_fail_dois)

In [71]:
with open("all_dois","wb") as p:
    pickle.dump(all_dois, p)

In [45]:
remainder_untested = remainder_df[~remainder_df.index.isin(all_dois)]

In [69]:
remainder_untested.shape

(80243, 19)

In [52]:
remainder_untested.Results.value_counts()

article_not_in_Scihub    9454
skipped                  3655
cant_read_pdf             242
direct_url_error           10
pdf_bytes_error             7
article_not_in Scihub       2
Name: Results, dtype: int64

In [55]:
remainder_untested[remainder_untested.Results.isna()].shape

(76588, 19)

The result above shows that remainder_untested has 89,958 articles remaining. However, due to the filtering conditions some are actually failed articles & will be moved to remainder_fail dataset.

The rest can be used in the following steps if necessary.


It is also seen that this step increased the number of untested artcl. count from 41K (pr_results_df) to 80K.


In [61]:
remainder_fail = pd.concat([remainder_fail, remainder_untested[~(remainder_untested.Results.isna()) | (remainder_untested.Results == "skipped")]])

remainder_untested = remainder_untested[(remainder_untested.Results.isna()) | (remainder_untested.Results == "skipped")]

All the remainder datasets are ready for future steps, so each will be saved accordingly.

In [70]:
with open("remainder_df","wb") as p:
    pickle.dump(remainder_df, p)

with open("remainder_succ","wb") as p:
    pickle.dump(remainder_succ, p)

with open("remainder_fail","wb") as p:
    pickle.dump(remainder_fail, p)

with open("remainder_untested","wb") as p:
    pickle.dump(remainder_untested, p)

### PART 4: Journal Datasets


In [4]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

with open("q1_jrnl_expladed", "rb") as fp:
    q1_jrnl_expladed = pickle.load(fp)

with open("remainder_untested", "rb") as fp:
    remainder_untested = pickle.load(fp)

In [6]:
q1_jrnl_df.Required_Count.sum()

29921

In [7]:
q1_jrnl_df.Current_Count.sum()

140993

In [17]:
len(q1_date_df)

156754

In [19]:
q1_jrnl_df["New_Current_Count"] = q1_date_df.groupby("JRNL_ID")["Dates"].count()

In [22]:
len(q1_jrnl_df[q1_jrnl_df.New_Current_Count < q1_jrnl_df.Current_Count])

297

In [23]:
len(q1_jrnl_df[q1_jrnl_df.New_Current_Count > q1_jrnl_df.Current_Count])

9652

It is seen that for 297 journals, fixing the "Received Date" issue caused a decrease in the number of articles. 

Whereas 9652 journals now have more articles after the partial fill run!



In [24]:
q1_jrnl_df.drop("Current_Count", axis=1, inplace=True)

In [26]:
q1_jrnl_df.drop("Required_Count", axis=1, inplace=True)

In [28]:
q1_jrnl_df.rename(columns={"New_Current_Count":"Current_Count"}, inplace=True)

q1_jnrl_df is now updated & can be SAVEd.

In [30]:
with open("q1_jrnl_df","wb") as p:
    pickle.dump(q1_jrnl_df, p)

Will go back to notebook 7b to re-create variables. 

__NOTE:__ During the re-run of 7b, it is seen that there are 26 articles that were written in the 20th century, but was published online recently; causing them to appear in the dataset, although they were published much earlier. These will be dropped from the dataset in the 7b nb, BUT will not SAVE q1_date_df there to avoid confusion.

An example: q1_date_df.loc["10.1080/00263206.2010.527121","direct_url"]

Therefore, these articles need to be dropped & q1_date_df will be updated HERE! 

__NOTE 2:__ Also need to check other edge cases for articles written before 2010 & after 2020!

In [ ]:
# Code for the case above:

#q1_date_df[q1_date_df.Dates.map(lambda x: x[0][-4:]).map(lambda x: not x.startswith("20"))]

During the analysis it is also seen that the current articles in q1_date_df requires Subj. Areas assigned that will also in line with the matrixes!

This will be the main focus TOMORROW!

The best strategy seems to be saving the pivots from 5b to be usedin this nb.

In [2]:

with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)


with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

### Part 4.5: __TTD:__

- add a JRNL_YEAR column to q1_date_df - DONE

- __q1_jnrl_df:__ - DONE
    - fix Current_Count
    - update 9+1 CR_columns - DONE
    - as we had retrials of some FAILED and all PARTIAL OK jrnls, need to compare with Req_Count to update Consider column
    - as a final touch re-write the conditions in the loop for Consider column


- add a S_A column 

- create pivot for currently SUCCESSFULL article count

- calc. the necessary article number

- re-distr. (WITH S_A info) the necessary count to q1_jrnl_df based on consider column (New_Req_Count)

- final extraction!

In [6]:
# TTD 1:

jrnl_year_dict = q1_jrnl_df.Year.to_dict()

q1_date_df["JRNL_YEAR"] = q1_date_df.JRNL_ID.map(jrnl_year_dict)

In [47]:
# TTD 1 - jrnl:

q1_jrnl_df.Current_Count = q1_jrnl_df.Current_Count.fillna(0)
q1_jrnl_df.Current_Count = q1_jrnl_df.Current_Count.astype(int)

ID
1_0        13
1_1         7
1_2         8
1_3        10
1_4         5
           ..
1_66911     3
1_66912     3
1_66913     0
1_66914     8
1_66915     0
Name: Current_Count, Length: 66916, dtype: int32

In [12]:
# TTD 2:
# Need to find the  most up-to-date Req. Count column:

# with open("q1_jrnl_df_25may", "rb") as fp:
#     q1_jrnl_df_25may = pickle.load(fp)

with open("q1_jrnl_df_21july", "rb") as fp:
    q1_jrnl_df_21july = pickle.load(fp)

q1_jrnl_df = q1_jrnl_df.merge(q1_jrnl_df_21july["Required_Count"], how="left", left_index=True, right_index=True)


In [52]:
# Also added the "old" Current_Count column to analyze the differences:
q1_jrnl_df_21july.rename(columns={"Current_Count":"Old_C_Count"},inplace=True)
q1_jrnl_df = q1_jrnl_df.merge(q1_jrnl_df_21july["Old_C_Count"], how="left", left_index=True, right_index=True)


In [57]:
len(q1_jrnl_df[q1_jrnl_df.Old_C_Count > q1_jrnl_df.Current_Count])

1939

In [58]:
len(q1_jrnl_df[(q1_jrnl_df.Old_C_Count > 0) & (q1_jrnl_df.Current_Count == 0)])

1642

It is seen that during the RECEIVED DATE only update (removed articles if only had Acc. Date in q1_date_df), 1939 jrnls in total seen a decrease in numbers. Out of which, 1642 does not have any remaining jrnls left.

They can be tested again to be sure, but will be skipped due to time limitations.



In [31]:
# Updating 9 + 1:
for id in ["1_18052","1_24838","1_24839","1_24840","1_24841","1_24842","1_24844","1_66638","1_66639"]:
    q1_jrnl_df.loc[id,"CR_Count"] = 0

q1_jrnl_df.loc["1_24843","CR_Count"] = 1


__Updating "Consider":__

- Consider is more up-to-date than Req_count
- Therefore if already Consider= False, it was skipped in partial fill & failed one of the conditions of Consider update in part 7c
- So need to update Consider == True & Req_Count > 0 articles! 

---


- All the analysis above shows that in the current q1_jrnl_df:
    - if a jrnl has Current_Count > 0, it can be considered for the last run.
    - However, do not forget that there are some Consider= False BUT Cur_Count > 0 jrnls. These are fine and will be considered in the pivot and not in the Consider.
    - 
    
- All in all we will be left with 32,734 jrnls that have at least 1 article in q1_date_df 
- And 32,688 jrnls that can be CONSIDERed in the next fillings!  


In [65]:
len(q1_jrnl_df[q1_jrnl_df.Consider == True])

34380

In [61]:
len(q1_jrnl_df[(q1_jrnl_df.Current_Count > 0)])

32734

In [62]:
len(q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.Current_Count > 0)])

32688

In [63]:
len(q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.Current_Count== 0)])

1692

__NOTE:__ 
1_42139, 1_14566 these journals USED TO have articles in the dataset, BUT 0 after RECEIVED_DATE ONLY update!

Code: q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.Current_Count== 0) & (q1_jrnl_df.Old_C_Count > 0)].sort_values("Old_C_Count", ascending=False)


In [68]:
# The update code: 

q1_jrnl_df.loc[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.Current_Count > 0), "New_Consider"] = True

In [72]:
# Finalize the dataset:

q1_jrnl_df.drop(["Consider","Required_Count","Old_C_Count"], axis=1, inplace= True)
q1_jrnl_df.rename(columns={"New_Consider": "Consider"}, inplace=True)

Consider col. is now updated. Both datasets should be up-to-date and ready for future proessing.

In [76]:
with open("q1_jrnl_df","wb") as fp:
   pickle.dump(q1_jrnl_df, fp)

In [75]:
q1_jrnl_df.head()

,ISSN,Year,Title,Sbj_Area,Total_Docs,Country,Region,Publisher,Coverage,SJR,...,Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,CR_Count,SH_Results,Current_Count,Consider
ID,,,,,,,,,,,,,,,,,,,,,
1_0,"(00011541, 15475905)",2013,AICHE Journal,"{'Biochemistry, Genetics and Molecular Biology...",422,United States,Northern America,Wiley-Blackwell,1955-2020,"1,053",...,956,17490,2697,928,"2,82","41,45",343,True,13,True
1_1,"(00011541, 15475905)",2014,AICHE Journal,"{'Biochemistry, Genetics and Molecular Biology...",359,United States,Northern America,Wiley-Blackwell,1955-2020,"1,066",...,1084,15256,3359,1058,"3,16","42,50",376,True,7,True
1_2,"(00011541, 15475905)",2019,AICHE Journal,"{'Biochemistry, Genetics and Molecular Biology...",335,United States,Northern America,Wiley-Blackwell,1955-2020,"0,947",...,1203,15387,4649,1189,"3,70","45,93",384,True,8,True
1_3,"(00011541, 15475905)",2018,AICHE Journal,"{'Biochemistry, Genetics and Molecular Biology...",379,United States,Northern America,Wiley-Blackwell,1955-2020,"0,978",...,1195,17900,4436,1181,"3,71","47,23",366,True,10,True
1_4,"(00011541, 15475905)",2020,AICHE Journal,"{'Biochemistry, Genetics and Molecular Biology...",374,United States,Northern America,Wiley-Blackwell,1955-2020,"0,958",...,1171,18063,4720,1155,"4,01","48,30",337,True,5,True


### Part 5: Adding S_A to q1_date_df


In [79]:
q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.Sbj_Area.map(len) == 1)].Current_Count.sum()

1    18549
2    10249
3     2871
4      796
5      174
6       47
7        2
Name: Sbj_Area, dtype: int64

In all posssible methods, we need to add the JRNL_S_A col. to q1_date_df:

In [91]:
jrnl_s_a_dict = q1_jrnl_df.Sbj_Area.map(lambda x: list(x.keys())).to_dict()

q1_date_df["JRNL_S_A"] = q1_date_df.JRNL_ID.map(jrnl_s_a_dict)

In [104]:
q1_date_df.JRNL_S_A.map(len).value_counts().sort_index()

1    56931
2    60616
3    25899
4     9986
5     2494
6      791
7       37
Name: JRNL_S_A, dtype: int64

In [106]:
def one_picker(sa_list):
    if len(sa_list) == 1:
        return sa_list[0]
    else:
        return np.nan

q1_date_df["S_A"] = q1_date_df.JRNL_S_A.map(one_picker)

In [117]:
pd.pivot_table(data=q1_date_df, columns="JRNL_YEAR", index="S_A", values="title", aggfunc="count")

JRNL_YEAR,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
S_A,,,,,,,,,,,
Agricultural and Biological Sciences,301,281,291,301,335,308,327,338,339,327,345
Arts and Humanities,189,192,215,202,271,211,247,252,232,216,243
"Biochemistry, Genetics and Molecular Biology",184,182,169,168,182,186,185,151,165,167,156
"Business, Management and Accounting",176,172,209,199,204,235,186,175,201,216,188
Chemical Engineering,102,75,85,81,113,73,109,78,100,107,74
Chemistry,128,135,139,162,154,168,155,134,122,131,131
Computer Science,188,183,181,171,168,206,188,185,189,166,168
Decision Sciences,6,8,10,7,3,7,12,7,7,7,10
Dentistry,226,169,264,224,190,240,308,255,358,318,310


In [12]:
with open("7b_\\q1_a_ss", "rb") as fp:
    q1_a_ss = pickle.load(fp)

The best way to assign S_A to q1_date_df would be to calculate the total number of eligible artlcs. for each cell ( so, creating a new pivot from exploded JRNL_S_A column) and dividing that with q1_a_ss to calculate the "S_A probability". 

#### Percentage Calculation:

In [125]:
q1_date_exploded = q1_date_df.explode("JRNL_S_A")

In [130]:
pd.pivot_table(data=q1_date_exploded, columns="JRNL_YEAR", index="JRNL_S_A", values="title", aggfunc="count")

JRNL_YEAR,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
JRNL_S_A,,,,,,,,,,,
Agricultural and Biological Sciences,1331,1378,1375,1375,1324,1370,1338,1313,1325,1266,1262
Arts and Humanities,873,887,903,839,875,888,869,862,876,909,836
"Biochemistry, Genetics and Molecular Biology",1506,1533,1508,1510,1510,1465,1423,1357,1318,1301,1285
"Business, Management and Accounting",932,957,948,997,947,988,1051,1068,1046,1103,1108
Chemical Engineering,1069,1124,1083,1050,1034,1060,991,966,984,1000,1004
Chemistry,1218,1253,1201,1213,1195,1150,1094,986,1008,1067,1127
Computer Science,1214,1238,1208,1186,1201,1226,1238,1204,1239,1286,1269
Decision Sciences,711,743,732,719,679,703,732,762,739,773,759
Dentistry,389,351,384,383,275,376,471,390,446,479,424


In [135]:
perc_pivot = (q1_a_ss / pd.pivot_table(data=q1_date_exploded, columns="JRNL_YEAR", index="JRNL_S_A", values="title", aggfunc="count")).round(3)

In [138]:
q1_date_exploded.drop("S_A", axis=1, inplace=True)

In [156]:
def perc_assign(row):
    # year = row["JRNL_YEAR"]
    # s_a = row["JRNL_S_A"]
    return perc_pivot.loc[row["JRNL_S_A"],row["JRNL_YEAR"]]
    

In [161]:
q1_date_exploded["Perc"] = q1_date_exploded.apply(perc_assign, axis=1)

In [168]:
q1_date_df["Perc_SA"] = q1_date_exploded.groupby(q1_date_exploded.index)["JRNL_S_A"].apply(list)

In [171]:
q1_date_df["Perc_Percs"] = q1_date_exploded.groupby(q1_date_exploded.index)["Perc"].apply(list)

q1_date_df["Perc_Percs"] = q1_date_df.Perc_Percs.map(lambda x: [round((item/sum(x)),3) for item in x])

In [183]:
q1_date_df["S_A_Percs"] = q1_date_df.apply(lambda x: dict(zip(x.Perc_SA, x.Perc_Percs)), axis=1)

In [185]:
q1_date_df["Chosen_S_A"] = q1_date_df.S_A_Percs.map(lambda x: random.choices(list(x.keys()), weights=x.values(), k=1))

q1_date_df["Chosen_S_A"] = q1_date_df.Chosen_S_A.map(lambda x: x[0])

In [13]:
sa_chosen_pivot = pd.pivot_table(data=q1_date_df, columns="JRNL_YEAR", index="Chosen_S_A", values="title", aggfunc="count")

In [195]:
# New Req_Count total:

(q1_a_ss - sa_chosen_pivot).applymap(lambda x: 0 if x<0 else x).sum().sum()

21856

In [16]:
missing_20k_pivot = (q1_a_ss - sa_chosen_pivot).applymap(lambda x: 0 if x<0 else x)

Will stop here for now! 

Will assume the S_A assigned to be TRUE!

The total number of necessary articles increased from around 15K for appr. 22K.

This is OK as the total number of articles will be somewhere around 180K, which was the plan before assignment.

Will save all variables

Tomorrow:

- will use sa_chosen_pivot to calc. Req_Count for the 180K run
- will save S_A this time beforehand
- arrange the 180K SumCetCet run 
- send codes!

- start working on author_df!

In [196]:
with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)

with open("q1_jrnl_df","wb") as p:
    pickle.dump(q1_jrnl_df, p)

with open("q1_date_exploded","wb") as p:
    pickle.dump(q1_date_exploded, p)

### Part 6: Missing 20K Distr. to q1_jrnl_df

In [2]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

with open("q1_date_exploded", "rb") as fp:
    q1_date_exploded = pickle.load(fp)



As decided before, we need to use q1_jrnl_df!!

Therefore, need to create a fitting Required_Count column. Also created a SA_list column for easy filtering.

In [181]:
q1_jrnl_df["Required_Count"] = q1_jrnl_df.Sbj_Area.map(dict)
q1_jrnl_df["Required_Count"] = q1_jrnl_df.Required_Count.map(lambda x: dict.fromkeys(x,int(0)))

q1_jrnl_df["SA_list"] = q1_jrnl_df.Required_Count.map(lambda x: list(x.keys()))

In [182]:
# Because of an error will add a Req_Count_sum column!

q1_jrnl_df["Required_Count_Sum"] = 0

Now we're ready to use missing_20k_pivot & modify loop from 7b to get the counts:

In [17]:
missing_20k_pivot

Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Sbj_Area,,,,,,,,,,,
Agricultural and Biological Sciences,0,0,0,0,0,0,0,0,0,0,0
Arts and Humanities,53,91,41,73,46,51,46,29,48,41,6
"Biochemistry, Genetics and Molecular Biology",9,0,3,0,0,0,0,39,25,17,32
"Business, Management and Accounting",31,59,39,42,46,31,52,54,31,33,46
Chemical Engineering,113,141,124,145,159,165,127,168,160,152,116
Chemistry,56,43,42,31,41,87,68,125,106,128,101
Computer Science,0,0,0,0,0,0,0,0,0,0,19
Decision Sciences,159,146,165,170,194,192,175,194,188,200,196
Dentistry,165,215,178,179,271,195,99,181,105,94,143


In [113]:
q1_jrnl_df_copy = q1_jrnl_df.copy()

In [183]:
for s_a, row in missing_20k_pivot.iterrows(): #iterate over rows
    print(s_a)
    for year, value in row.items():
        print(year)
        if value <= 0:
            pass
        else:
            for i in range(value):
                # select one: 
                sampl_index = q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.CR_Count > q1_jrnl_df.Required_Count_Sum) & (q1_jrnl_df.SA_list.map(lambda x: s_a in x)) & (q1_jrnl_df.Year == year)].sample(1).index.item()
        

                # jrnl_rc = sampl_index["Required_Count"].item()


                # sa_count = jrnl_rc[s_a] 
                # sa_count += 1
                # jrnl_rc[s_a] = sa_count
                # jrnl_rc_sum += 1

                q1_jrnl_df.loc[sampl_index,"Required_Count"][s_a] += 1
                q1_jrnl_df.loc[sampl_index,"Required_Count_Sum"] += 1
                

                


Agricultural and Biological Sciences
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Arts and Humanities
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Biochemistry, Genetics and Molecular Biology
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Business, Management and Accounting
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Chemical Engineering
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Chemistry
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Computer Science
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Decision Sciences
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Dentistry
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Earth and Planetary Sciences
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Economics, Econometrics and Finance
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Energy
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Engineering
2010
2011
2012
2013
2014
2015
2016
2017
20

In [204]:
with open("q1_jrnl_df","wb") as p:
    pickle.dump(q1_jrnl_df, p)

The loop FINALLY succeded and q1_jrnl_df is now ready for the CR & SH runs. However, berfore the CR run we need to check if there are any eligible (meta) articles from remainder_untested.

### Part 7: Reaching 180K

#### remainder_untested check:

In [206]:
with open("remainder_untested", "rb") as fp:
    remainder_untested = pickle.load(fp)


In [5]:
# Small Corrections:
remainder_untested = remainder_untested[remainder_untested.JRNL_ID.notna()]

In [213]:
req_jrnl_ids = q1_jrnl_df[q1_jrnl_df.Required_Count_Sum != 0].index.tolist()

In [220]:
len(remainder_untested[remainder_untested.JRNL_ID.map(lambda x: x in req_jrnl_ids)])

40707

Initial analysis show that 40K articles from remainder_untested is actually eligible!

But need to check with the Req_Count in mind as well!

In [228]:
q1_jrnl_df["Untested_Count"] = remainder_untested.groupby("JRNL_ID")["title"].count()

In [231]:
q1_jrnl_df.Untested_Count.fillna(0, inplace = True)

In [34]:
def ok_untested(row):
    u_c = row.Untested_Count
    rcs = row.Required_Count_Sum

    if rcs == 0:
        return 0
    elif rcs >= u_c:
        return u_c
    elif u_c > rcs:
        return rcs


In [35]:
q1_jrnl_df["Useful_Untested"] = q1_jrnl_df.apply(ok_untested, axis=1)

In [36]:
q1_jrnl_df.Useful_Untested.sum()

12726

In [244]:
with open("q1_jrnl_df","wb") as p:
    pickle.dump(q1_jrnl_df, p)

with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)

Around 12K can be used!

Now need to create a SH loop for them and modify the loop from 7b for the remaining 10K!

In [2]:
with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)


with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)



#### Creating untested_df SH run: 

In [22]:
# Preparing remainder_untested:

with open("remainder_untested", "rb") as fp:
    remainder_untested = pickle.load(fp)

# There was also a small correction yesterday, seen above

remainder_untested.drop(["published", "direct_url", "Dates", "Keywords", "Affiliation", "reference-count", "Results"], axis=1, inplace=True)


In [15]:
# Small Corrections on q1_date_df:
q1_date_df.drop(["S_A", "Perc_SA","Perc_Percs", "S_A_Percs"],axis=1, inplace=True)

In [38]:
# Small Corrections on q1_jrnl_df:
q1_jrnl_df["Useful_Untested"] = q1_jrnl_df.Useful_Untested.astype(int)

In [42]:
# Preparing sel_untested_df:

untested_jrnl_dict = q1_jrnl_df[q1_jrnl_df.Useful_Untested > 0]["Useful_Untested"].to_dict()

sel_untested_df = pd.DataFrame()

for key, value in untested_jrnl_dict.items():
    try:
        sampl_artcls = remainder_untested[remainder_untested.JRNL_ID == key].sample(value+2)
    except ValueError:
        sampl_artcls = remainder_untested[remainder_untested.JRNL_ID == key].sample(value)
    
    
    sel_untested_df = pd.concat([sel_untested_df,sampl_artcls])

with open("sel_untested_df","wb") as p:
    pickle.dump(sel_untested_df, p)


sel_untested_df is now ready for SH run!!!

Can use the SumCetCet .py code from 7c! 

#### Selected_Untested Run (Ogi):

In [ ]:
# Copied from 7c & modified:

sh_h = SciHub_HydroPump()


doi_list = sel_untested_df.DOI.tolist()

sel_untested_dates = list() 


for doi in doi_list:
    pdf_url, date, kw = sh_h.get_dates_kws(doi)

    sel_untested_dates.append(dict(DOI = doi, direct_url = pdf_url, Dates = date, Keywords = kw))

# with open("q1_date_fe_cet","wb") as fp:
#     pickle.dump(q1_date_fe_cet, fp)

Created a seperate nb under 7e folder, currently running!!

---

In the meantime will also create the necessary cols & code for the remaining 9K 

In [59]:
# Preparing new_required_df:

q1_jrnl_df["New_Required"] = q1_jrnl_df.Required_Count_Sum - q1_jrnl_df.Useful_Untested

new_required_df = q1_jrnl_df[q1_jrnl_df.New_Required > 0]

with open("new_required_df","wb") as p:
    pickle.dump(new_required_df, p)

In [78]:
# Need to create a doi list incl. q1_date_df & sel_untested_df:

doi_skip_list = q1_date_df.index.to_list() + sel_untested_df.index.to_list()

with open("doi_skip_list","wb") as p:
    pickle.dump(doi_skip_list, p)

#### New_Required Run (Sum):

In [ ]:
# TAKEN FROM 7C (partial fill run) - CR + SH RUN for new_required_df:

sh_h = SciHub_HydroPump()

mail = 'oguzkokes@gmail.com'
my_etiquette = Etiquette('Analysing Publishing Delay in Academic Journals', 'v2.0', 'https://github.com/Spidey0023/THEsis-Codes', mail)
ogi_works = Works(etiquette=my_etiquette)

partial_df = new_required_df[["ISSN","Year", "New_Required","CR_Count"]]
partial_df["ISSN"] = partial_df.ISSN.map(list)
pr_dict = partial_df.to_dict("index")

new_req_cr_results = []

new_req_sh_succ = list() 
new_req_sh_fail = list()

useful_cols = ['DOI', 'references-count', 'publisher', 'published-print', 'is-referenced-by-count', 'title', 'author', 'published-online', 'container-title', 'issued', 'ISSN', 'subject']

print("Starting CR Phase:")


for key in pr_dict:
    issn = pr_dict[key]["ISSN"]
    year = pr_dict[key]["Year"]

    rc = pr_dict[key]["New_Required"]
    crc = pr_dict[key]["CR_Count"]
    min_rc = min(rc+2, crc)

    loop_meta = [w for w in ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").sample(int(min_rc)).select(useful_cols)]
    loop_meta = [dict(item, JRNL_ID=key) for item in loop_meta if item["DOI"] not in doi_skip_list[key]]
    new_req_cr_results.extend(loop_meta)

    req_check = 0

    for artcl in loop_meta:
        if req_check == rc:
            break
        else:
            doi = artcl["DOI"]
            pdf_url, date, kw = sh_h.get_dates_kws(doi)
            if type(date) == list:
                new_req_sh_succ.append(dict(DOI = doi, direct_url = pdf_url, Dates = date, Keywords = kw))
                req_check += 1
            else:
                new_req_sh_fail.append(dict(DOI = doi, direct_url = pdf_url, Dates = date, Keywords = kw))

In [82]:
with open("q1_jrnl_df","wb") as p:
    pickle.dump(q1_jrnl_df, p)

with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)

Sent the files to Summan!

Both loops are running which will hopefully completely end this part of the project!

__Remaining TTDs:__

- author_df & Scopus API
- analysis & graphing
- writing the report

- FIN!

### Part 8: SumOgi Analysis

While waiting for the q1_first_author_df run to end, will analyse the SumOgi results to understand if we have finally reached the required number of Q1 articles.

#### 8.1 - New Required Analysis:

In [2]:
# new_required_df - Sum:

with open("7e_variables/new_required_df_run_Sum/new_req_suc_df", "rb") as fp:
    new_req_suc_df = pickle.load(fp)

with open("7e_variables/new_required_df_run_Sum/new_req_fail_df", "rb") as fp:
    new_req_fail_df = pickle.load(fp)


In [16]:
new_req_fail_df = pd.concat([new_req_fail_df,new_req_suc_df[new_req_suc_df.Dates.map(lambda x: type(x[0]) != str)]], ignore_index=True)

In [18]:
new_req_suc_df = new_req_suc_df[new_req_suc_df.Dates.map(lambda x: type(x[0]) == str)]

In [19]:
# TTD 1:
# Taken from 7c:

# 1.3 - Filtering to only successful artcls:
# q1_date_pr = pr_results_df[pr_results_df.Dates.map(type)==list]
new_req_suc_df.set_index("DOI", drop=True, inplace=True)

# 1.4 - Feature Eng. : 
def earlier_date_part(row):
    try:
        pp_year = row["published-print"]["date-parts"][0][0]
    except TypeError:
        pp_year = None

    try:   
        po_year = row["published-online"]["date-parts"][0][0]
    except TypeError:
        po_year = None

    if po_year is None:
        return row["published-print"]

    elif pp_year is None:
        return row["published-online"]

    else:
        if pp_year<po_year:
            return row["published-print"]
        elif po_year<pp_year:
            return row["published-online"]
        else:
            try:
                pp_month = row["published-print"]["date-parts"][0][1]
            except IndexError:
                pp_month = 12
            try:
                po_month = row["published-online"]["date-parts"][0][1]
            except IndexError:
                po_month = 12
            if pp_month<po_month:
                return row["published-print"]
            elif po_month<pp_month:
                return row["published-online"]
            else:
                return row["published-online"]


new_req_suc_df.loc[:,"published"] = new_req_suc_df.apply(earlier_date_part, axis=1)

new_req_suc_df = new_req_suc_df[["JRNL_ID", "ISSN", "issued", "container-title", "publisher", "title", "author", "subject", "references-count", "is-referenced-by-count", "published", "published-print", "published-online", "direct_url", "Dates", "Keywords"]]


<ipython-input-19-14a83e6a5e8d>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_req_suc_df.loc[:,"published"] = new_req_suc_df.apply(earlier_date_part, axis=1)


new_req_suc_df is ready to be added to q1_date_df 

#### 8.2 - Selected Untested Analysis:

In [3]:
# untested_dates_df - Ogi:

with open("7e_variables/sel_untested_dates_run_Ogi/sel_untested_dates", "rb") as fp:
    sel_untested_dates = pickle.load(fp)

with open("7e_variables/sel_untested_dates_run_Ogi/sel_untested_df", "rb") as fp:
    sel_untested_df = pickle.load(fp)

In [26]:
sel_untested_df = sel_untested_df.merge(pd.DataFrame(sel_untested_dates), how="left", left_index=True, right_on="DOI")

In [29]:
sel_untested_succ = sel_untested_df[sel_untested_df.Dates.map(lambda x: type(x) == list)]

In [32]:
sel_untested_succ = sel_untested_succ[sel_untested_succ.Dates.map(lambda x: type(x[0]) == str)]

In [33]:
# TTD 1:
# Taken from 7c:

# 1.3 - Filtering to only successful artcls:
# q1_date_pr = pr_results_df[pr_results_df.Dates.map(type)==list]
sel_untested_succ.set_index("DOI", drop=True, inplace=True)


sel_untested_succ.loc[:,"published"] = sel_untested_succ.apply(earlier_date_part, axis=1)

sel_untested_succ = sel_untested_succ[["JRNL_ID", "ISSN", "issued", "container-title", "publisher", "title", "author", "subject", "references-count", "is-referenced-by-count", "published", "published-print", "published-online", "direct_url", "Dates", "Keywords"]]


sel_untested_succ is ready as well.

#### 8.3 - new_additions_df

In [4]:
with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

In [39]:
new_additions_df = pd.concat([new_req_suc_df,sel_untested_succ],verify_integrity=True)

In [ ]:
# To ease memory usage:

del new_req_fail_df
del new_req_suc_df
del sel_untested_dates
del sel_untested_succ

Need to create a function to assign Sbj_areas to new_additions_df. However, while doing so, do fn. must also take into account extra articles! 

First, distr. Required_Count col. & if any extra artcls. remaining, randomly assign from SA_list

In [51]:
# Will restart pc & so saving all the modified variables:

with open("new_additions_df","wb") as p:
    pickle.dump(new_additions_df, p)

In [2]:
# After restart:

with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

with open("new_additions_df", "rb") as fp:
    new_additions_df = pickle.load(fp)

##### 8.3.1 - Testing:

In [187]:
# Version 1 - Function:

def new_add_sa_mapper(jrnl_id):
    rc_dict = q1_jrnl_df.loc[jrnl_id,"Required_Count_dist"]
    if sum(rc_dict.values()) == 0:
        return choice(list(rc_dict.keys()))
    else:
        chosen_sa = choice([k for k,v in rc_dict.items() if v>0])
        q1_jrnl_df.loc[jrnl_id,"Required_Count_dist"][chosen_sa] -= 1
        return chosen_sa
    

The function works BUT does not update q1_jrnl_df.

Therefore will create a loop instead of a function.

In [229]:
testo_df = new_additions_df.sample(10)

In [233]:
# Version 2 - iterrows:

for id, row in testo_df.iterrows():
    jrnl_id = row["JRNL_ID"]
    rc_dict = q1_jrnl_df.loc[jrnl_id,"Required_Count_dist"]
    
    print(type(rc_dict))
    print(list(rc_dict.values()))
    print([k for k,v in rc_dict.items() if v>0])

    if sum(list(rc_dict.values())) == 0:
        testo_df.loc[id,"Chosen_SA"] = choice(list(rc_dict.keys()))

    else:
        chosen_sa = choice([k for k,v in rc_dict.items() if v>0])
        q1_jrnl_df.loc[jrnl_id,"Required_Count_dist"][chosen_sa] -= 1
        testo_df.loc[id,"Chosen_SA"] = chosen_sa


<class 'dict'>
[1, 0]
['Physics and Astronomy']
<class 'dict'>
[0, 0, 2]
['Nursing']
<class 'dict'>
[5, 0]
['Immunology and Microbiology']
<class 'dict'>
[1, 0]
['Neuroscience']
<class 'dict'>
[2]
['Energy']
<class 'dict'>
[0, 0, 1]
['Physics and Astronomy']
<class 'dict'>
[0, 0, 1, 0]
['Economics, Econometrics and Finance']
<class 'dict'>
[0, 2]
['Arts and Humanities']
<class 'dict'>
[0, 0, 3]
['Nursing']
<class 'dict'>
[1]
['Environmental Science']


Also does not update q1_jnrl_df

In [210]:
testo_dict = testo_df["JRNL_ID"].to_dict()

In [211]:
# Version 3 - dict:

for doi in testo_dict:
    jrnl_id = testo_dict[doi]
    rc_dict = q1_jrnl_df.loc[jrnl_id,"Required_Count_dist"]

    if sum(list(rc_dict.values())) == 0:
        testo_df.loc[doi,"Chosen_SA"] = choice(list(rc_dict.keys()))

    else:
        chosen_sa = choice([k for k,v in rc_dict.items() if v>0])
        # new_val = rc_dict[chosen_sa] - 1
        # rc_dict[chosen_sa] = new_val
        q1_jrnl_df.loc[jrnl_id,"Required_Count_dist"][chosen_sa] -= 1
        testo_df.loc[doi,"Chosen_SA"] = chosen_sa


In [234]:
q1_jrnl_df.loc["1_63339","Required_Count"]

{'Physics and Astronomy': 0, 'Biochemistry, Genetics and Molecular Biology': 0}

In [235]:
q1_jrnl_df.loc["1_63339","Required_Count"]

q1_jrnl_df.loc["1_34423", "Required_Count_dist"]

{'Physics and Astronomy': 0, 'Biochemistry, Genetics and Molecular Biology': 0}

After the tests it is seen that actually iterrows method works, however Required_Sum column is also updated each time.

As a result, both columns had he same values although they were both different than before.

Will try to fix this issue and use the iterrows version!

##### 8.3.2 - Full Loop:

In [4]:
for id, row in new_additions_df.iterrows():
    jrnl_id = row["JRNL_ID"]
    rc_dict = q1_jrnl_df.loc[jrnl_id,"Required_Count"]
    
    # print(type(rc_dict))
    # print(list(rc_dict.values()))
    # print([k for k,v in rc_dict.items() if v>0])

    if sum(list(rc_dict.values())) == 0:
        new_additions_df.loc[id,"Chosen_SA"] = choice(list(rc_dict.keys()))

    else:
        chosen_sa = choice([k for k,v in rc_dict.items() if v>0])
        q1_jrnl_df.loc[jrnl_id,"Required_Count"][chosen_sa] -= 1
        new_additions_df.loc[id,"Chosen_SA"] = chosen_sa


In [8]:
q1_jrnl_df["New_Additions"] = new_additions_df.groupby("JRNL_ID")["title"].count()

q1_jrnl_df.New_Additions.fillna(0, inplace=True)

q1_jrnl_df["New_Additions"] = q1_jrnl_df.New_Additions.astype(int)

In [18]:
# Number of jrnls with LESS than Req. retr:

len(q1_jrnl_df[q1_jrnl_df.Required_Count_Sum > q1_jrnl_df.New_Additions])

2801

In [19]:
# Number of jrnls with MORE than Req. retr:

len(q1_jrnl_df[q1_jrnl_df.Required_Count_Sum < q1_jrnl_df.New_Additions])

4402

In [20]:
q1_date_df_copy = q1_date_df.copy()

In [22]:
# Small modifications on q1_date_df:

q1_date_df.drop(["Affiliation","JRNL_YEAR","JRNL_S_A"], axis=1, inplace=True)

q1_date_df.rename(columns={"Chosen_S_A":"Chosen_SA"}, inplace=True)

In [29]:
# Combining all:
q1_date_df = pd.concat([q1_date_df,new_additions_df],verify_integrity=True)

#### 8.4 - Pivot Analysis:

In [31]:
with open("7b_variables/q1_a_ss", "rb") as fp:
    q1_a_ss = pickle.load(fp)

In [33]:
jrnl_year_dict = q1_jrnl_df.Year.to_dict()

q1_date_df["JRNL_YEAR"] = q1_date_df.JRNL_ID.map(jrnl_year_dict)

In [35]:
after_new_add_pivot = pd.pivot_table(data=q1_date_df, columns="JRNL_YEAR", index="Chosen_SA", values="title", aggfunc="count")

In [40]:
(q1_a_ss - after_new_add_pivot).applymap(lambda x: x if x>0 else 0)

Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Sbj_Area,,,,,,,,,,,
Agricultural and Biological Sciences,0,0,0,0,0,0,0,0,0,0,0
Arts and Humanities,0,0,0,0,0,0,0,0,0,0,0
"Biochemistry, Genetics and Molecular Biology",0,0,0,0,0,0,0,0,0,0,0
"Business, Management and Accounting",0,0,0,0,0,0,0,0,0,0,0
Chemical Engineering,0,0,0,0,0,18,6,39,4,0,19
Chemistry,0,0,0,0,0,0,0,0,0,21,0
Computer Science,0,0,0,0,0,0,0,0,0,0,0
Decision Sciences,31,18,29,29,48,34,29,36,55,44,45
Dentistry,73,102,81,98,152,97,49,93,36,30,77


In [39]:
(q1_a_ss - after_new_add_pivot).applymap(lambda x: x if x>0 else 0).sum().sum()

3170

In [41]:
with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)

with open("q1_jrnl_df","wb") as p:
    pickle.dump(q1_jrnl_df, p)

with open("q1_date_df_copy","wb") as p:
    pickle.dump(q1_date_df, p)

with open("q1_jrnl_df_copy","wb") as p:
    pickle.dump(q1_jrnl_df, p)

#### 8.5 - 5K loop preparations:

__TTD:__

- update q1_jrnl_df
- update all_dois list with Sum & Ogi results
- modify & use 20K loop from above
- prepare Sum nb

In [2]:
# After restart:

with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

with open("7b_variables/q1_a_ss", "rb") as fp:
    q1_a_ss = pickle.load(fp)

##### 8.5.1 - q1_jrnl_df updates:

In [6]:
q1_jrnl_df.columns

Index(['ISSN', 'Year', 'Title', 'Sbj_Area', 'Total_Docs', 'Country', 'Region',
       'Publisher', 'Coverage', 'SJR', 'H index', 'Total Docs. (3years)',
       'Total Refs.', 'Total Cites (3years)', 'Citable Docs. (3years)',
       'Cites / Doc. (2years)', 'Ref. / Doc.', 'CR_Count', 'SH_Results',
       'Current_Count', 'Consider', 'Required_Count', 'SA_list',
       'Required_Count_Sum', 'Untested_Count', 'Useful_Untested',
       'New_Required', 'New_Additions'],
      dtype='object')

In [7]:
q1_jrnl_df.drop(["Required_Count",'Required_Count_Sum', 'Untested_Count', 'Useful_Untested', 'New_Required', 'New_Additions'], axis=1, inplace=True)

q1_jrnl_df["Current_Count"] = q1_date_df.groupby("JRNL_ID")["title"].count()
q1_jrnl_df["Current_Count"] = q1_jrnl_df.Current_Count.fillna(0).astype(int)

##### 8.5.2 - remainder_df & untested_df updates:

It is decided that there will be 3 article datasets from now on:

__1-__ q1_date-df  -> MY BABY BOY

__2-__ untested_df -> CR loop complete, no SH loop

__3-__ remainder_df -> all failed articles are collected here


In [51]:
# Step 1 - create untested_df from remainder_untested:

with open("remainder_untested", "rb") as fp:
    remainder_untested = pickle.load(fp)

with open("7e_variables/sel_untested_dates_run_Ogi/sel_untested_df", "rb") as fp:
    sel_untested_df = pickle.load(fp)


In [54]:
untested_df = remainder_untested[~remainder_untested.index.isin(sel_untested_df.index.to_list())]

In [55]:
# Correction from part 7 (remainder_untested was fixed but not saved, only sel_untested_df was saved, so have to repeat)

untested_df.drop(["published", "direct_url", "Dates", "Keywords", "Affiliation", "reference-count", "Results"], axis=1, inplace=True)


<ipython-input-55-864c91b238a8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  untested_df.drop(["published", "direct_url", "Dates", "Keywords", "Affiliation", "reference-count", "Results"], axis=1, inplace=True)


In [104]:
untested_df = untested_df[~untested_df.index.isin(q1_date_df.index.to_list())]

Now, untested_df is up-to-date and ready to be saved

In [74]:
# Step 2: remainder_df

with open("remainder_df", "rb") as fp:
    remainder_df = pickle.load(fp)

with open("7e_variables/new_required_df_run_Sum/new_req_suc_df", "rb") as fp:
    new_req_suc_df = pickle.load(fp)

with open("7e_variables/new_required_df_run_Sum/new_req_fail_df", "rb") as fp:
    new_req_fail_df = pickle.load(fp)


In [89]:
remainder_df = remainder_df[~remainder_df.index.duplicated(keep='last')]

new_req_fail_df.set_index("DOI", inplace=True)
remainder_df = remainder_df[~remainder_df.index.isin(new_req_fail_df.index.to_list())]

new_req_suc_df.set_index("DOI", inplace=True)
remainder_df = remainder_df[~remainder_df.index.isin(new_req_suc_df.index.to_list())]

New required run used doi_skip_list var. during loop. This caused some artcls. in remainderd_df to be re-downloaded.

To only keep the most up-to-date info, will drop them first & concat all.

In [96]:
remainder_df = pd.concat([remainder_df,new_req_fail_df,new_req_suc_df], verify_integrity=True)

In [100]:
remainder_df = remainder_df[~remainder_df.index.isin(q1_date_df.index.to_list())]

In [106]:
remainder_df = remainder_df[~remainder_df.index.isin(untested_df.index.to_list())]

In [107]:
len(remainder_df)

142308

Now, remainder_df is also up-to-date and ready to be saved

In [108]:
del sel_untested_df
del remainder_untested
del new_req_fail_df
del new_req_suc_df
del doi_skip_list

In [109]:
# Added q1 to their names:

with open("q1_remainder_df","wb") as p:
    pickle.dump(remainder_df, p)

with open("q1_untested_df","wb") as p:
    pickle.dump(untested_df, p)


### Part 9: Distributing 5K articles to q1_jrnl_df & 5K run mock-up:

In the future can simply follow the steps from this part to collect articles as necessary!

#### 9.1 - Distr. articles:

In [110]:
after_new_add_pivot = pd.pivot_table(data=q1_date_df, columns="JRNL_YEAR", index="Chosen_SA", values="title", aggfunc="count")

In [111]:
missing_5k_pivot = (q1_a_ss - after_new_add_pivot).applymap(lambda x: x if x>0 else 0)

In [114]:
q1_jrnl_df["Required_Count"] = q1_jrnl_df.Sbj_Area.map(dict)
q1_jrnl_df["Required_Count"] = q1_jrnl_df.Required_Count.map(lambda x: dict.fromkeys(x,int(0)))

q1_jrnl_df["SA_list"] = q1_jrnl_df.Required_Count.map(lambda x: list(x.keys()))

q1_jrnl_df["Required_Count_Sum"] = 0

In [116]:
for s_a, row in missing_5k_pivot.iterrows(): #iterate over rows
    print(s_a)
    for year, value in row.items():
        print(year)
        if value == 0:
            pass
        else:
            for i in range(int(value*1.5)):
                # select one: 
                sampl_index = q1_jrnl_df[(q1_jrnl_df.Consider == True) & (q1_jrnl_df.CR_Count > (q1_jrnl_df.Required_Count_Sum + q1_jrnl_df.Current_Count)) & (q1_jrnl_df.SA_list.map(lambda x: s_a in x)) & (q1_jrnl_df.Year == year)].sample(1).index.item()
        

                # jrnl_rc = sampl_index["Required_Count"].item()


                # sa_count = jrnl_rc[s_a] 
                # sa_count += 1
                # jrnl_rc[s_a] = sa_count
                # jrnl_rc_sum += 1

                q1_jrnl_df.loc[sampl_index,"Required_Count"][s_a] += 1
                q1_jrnl_df.loc[sampl_index,"Required_Count_Sum"] += 1
                

                


Agricultural and Biological Sciences
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Arts and Humanities
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Biochemistry, Genetics and Molecular Biology
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Business, Management and Accounting
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Chemical Engineering
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Chemistry
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Computer Science
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Decision Sciences
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Dentistry
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Earth and Planetary Sciences
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Economics, Econometrics and Finance
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Energy
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Engineering
2010
2011
2012
2013
2014
2015
2016
2017
20

In [118]:
missing_5k_pivot.sum().sum()

3170

In [117]:
q1_jrnl_df.Required_Count_Sum.sum()

4733

#### 9.2 - Creating all_dois:

In [131]:
all_dois_5k = set(q1_date_df.index.to_list() + remainder_df.index.to_list())

In [139]:
with open("all_dois_5k","wb") as p:
    pickle.dump(all_dois_5k, p)

#### 9.3 - 5K Loop Mock-up:

In [134]:
required_5k_df = q1_jrnl_df[q1_jrnl_df.Required_Count_Sum > 0]

In [140]:
with open("required_5k_df","wb") as p:
    pickle.dump(required_5k_df, p)

In [ ]:
with open("required_5k_df", "rb") as fp:
    required_5k_df = pickle.load(fp)

with open("all_dois_5k", "rb") as fp:
    all_dois_5k = pickle.load(fp)

In [ ]:

sh_h = SciHub_HydroPump()

mail = 'sduzenli@yandex.com.tr'

my_etiquette = Etiquette('Analysing Publishing Delay in Academic Journals', 'v2.0', 'https://github.com/Spidey0023/THEsis-Codes', mail)
ogi_works = Works(etiquette=my_etiquette)



partial_df = new_required_df[["ISSN","Year", "New_Required","CR_Count"]]

partial_df.loc[:,"ISSN"] = partial_df.ISSN.map(list)
pr_dict = partial_df.to_dict("index")

new_req_cr_results = list()
new_req_sh_results = list() 


useful_cols = ['DOI', 'references-count', 'publisher', 'published-print', 'is-referenced-by-count', 'title', 'author', 'published-online', 'container-title', 'issued', 'ISSN', 'subject']

for key in pr_dict:
    issn = pr_dict[key]["ISSN"]
    year = pr_dict[key]["Year"]

    rc = pr_dict[key]["New_Required"]
    crc = pr_dict[key]["CR_Count"]
    min_rc = min(rc+3, crc)

    loop_meta = [w for w in ogi_works.filter(issn=issn, from_pub_date=year, until_pub_date=year, type="journal-article").sample(int(min_rc)).select(useful_cols)]
    loop_meta = [dict(item, JRNL_ID=key) for item in loop_meta if item["DOI"] not in doi_skip_list]
    new_req_cr_results.extend(loop_meta)

    for artcl in loop_meta:
        doi = artcl["DOI"]
        pdf_url, date, kw = sh_h.get_dates_kws(doi)
        new_req_sh_results.append(dict(DOI = doi, direct_url = pdf_url, Dates = date, Keywords = kw))


In [141]:
with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)

with open("q1_jrnl_df","wb") as p:
    pickle.dump(q1_jrnl_df, p)

In [2]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)


In [4]:
with open("q1_date_df_Aug6","wb") as p:
    pickle.dump(q1_date_df, p)

In [5]:
with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)


In [3]:
with open("author_out_list_loop_save", "rb") as fp:
    author_out_list_loop_save = pickle.load(fp)


In [5]:
with open("q1_first_author_df", "rb") as fp:
    q1_first_author_df = pickle.load(fp)


In [4]:
pd.DataFrame(author_out_list_loop_save)

,author_id,country,given,family,doc_count
0,aut_0,"[China, China, China, Hong Kong]","[Jing, Jingheng, Jing, (Luna) Jing]","[Cai, Cai, Cai, Cai]","[23, 22, 1, 1]"
1,aut_1,"[United States, United States, United States, ...","[Qing, Qing, (Grace) Qing, Grace Qing]","[Hao, Hao, Hao, Hao]","[86, 11, 2, 1]"
2,aut_2,"[China, China, China, China, China, China, Chi...","[Yongquan, Yonghong, Yong, Dayong, Yongwu, Yon...","[Zhou, Zhou, Zhou, Zhou, Zhou, Zhou, Zhou, Zho...","[286, 220, 193, 170, 164, 151, 100, 92, 74, 64..."
3,aut_3,"[Italy, Italy, France, Nigeria, Malaysia, Saud...","[Andrea, Alessandro, Ado Adamou Abba, Hafizull...","[Abba, Abbà, Ari, Ahmed, Haruna, Abba, Abba, A...","[75, 65, 44, 19, 17, 15, 8, 3]"
4,aut_4,[Italy],[Antonella],[Agodi],[234]
...,...,...,...,...,...
92712,aut_92717,[United States],[Michael J.],[Jung],[10]
92713,aut_92718,[United States],[Michael J.],[Jurynec],[24]
92714,aut_92719,"[United States, United States, United States]","[John Michael, John Michael, Michael J.]","[Kane, Kane, Kane]","[511, 338, 87]"
92715,aut_92720,"[United States, United States]","[Michael J., Joseph Michael]","[Keenan, Keenan]","[79, 41]"


#### 9.4 - 5K Run Results & Analysis

In [2]:
with open("new_req_cr_results_final", "rb") as fp:
    new_req_cr_results_final = pickle.load(fp)

with open("new_req_sh_results_final", "rb") as fp:
    new_req_sh_results_final = pickle.load(fp)


In [29]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

In [17]:
# Create df (CR + SH):
k5k_new_df = pd.DataFrame(new_req_cr_results_final).merge(pd.DataFrame(new_req_sh_results_final), how="inner", on="DOI")

# to be added to date_df:
k5k_new_succ = k5k_new_df[k5k_new_df.Dates.map(lambda x: type(x) == list)]
k5k_new_succ = k5k_new_succ[k5k_new_succ.Dates.map(lambda x: type(x[0]) == str)]

# to be added to remainder_df:
k5k_new_rem_df = k5k_new_df[~k5k_new_df.DOI.isin(k5k_new_succ.index.to_list())]


In [28]:
del new_req_cr_results_final
del new_req_sh_results_final

In [20]:
def earlier_date_part(row):
    try:
        pp_year = row["published-print"]["date-parts"][0][0]
    except TypeError:
        pp_year = None

    try:   
        po_year = row["published-online"]["date-parts"][0][0]
    except TypeError:
        po_year = None

    if po_year is None:
        return row["published-print"]

    elif pp_year is None:
        return row["published-online"]

    else:
        if pp_year<po_year:
            return row["published-print"]
        elif po_year<pp_year:
            return row["published-online"]
        else:
            try:
                pp_month = row["published-print"]["date-parts"][0][1]
            except IndexError:
                pp_month = 12
            try:
                po_month = row["published-online"]["date-parts"][0][1]
            except IndexError:
                po_month = 12
            if pp_month<po_month:
                return row["published-print"]
            elif po_month<pp_month:
                return row["published-online"]
            else:
                return row["published-online"]

In [21]:
# Eng. on new dataset:

k5k_new_succ.set_index("DOI", drop=True, inplace=True)
k5k_new_succ.loc[:,"published"] = k5k_new_succ.apply(earlier_date_part, axis=1)
k5k_new_succ = k5k_new_succ[["JRNL_ID", "ISSN", "issued", "container-title", "publisher", "title", "author", "subject", "references-count", "is-referenced-by-count", "published", "published-print", "published-online", "direct_url", "Dates", "Keywords"]]
k5k_new_succ.loc[:,"JRNL_YEAR"] = k5k_new_succ.JRNL_ID.map(q1_jrnl_df.Year.to_dict())

In [32]:
for id, row in k5k_new_succ.iterrows():
    jrnl_id = row["JRNL_ID"]
    rc_dict = q1_jrnl_df.loc[jrnl_id,"Required_Count"]
    
    # print(type(rc_dict))
    # print(list(rc_dict.values()))
    # print([k for k,v in rc_dict.items() if v>0])

    if sum(list(rc_dict.values())) == 0:
        k5k_new_succ.loc[id,"Chosen_SA"] = choice(list(rc_dict.keys()))

    else:
        chosen_sa = choice([k for k,v in rc_dict.items() if v>0])
        q1_jrnl_df.loc[jrnl_id,"Required_Count"][chosen_sa] -= 1
        k5k_new_succ.loc[id,"Chosen_SA"] = chosen_sa


<ipython-input-32-50bf34492cb8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k5k_new_succ.loc[id,"Chosen_SA"] = chosen_sa


In [35]:
# Small modifitcation on jrnl_df & saving:

q1_jrnl_df["New_Additions"] = k5k_new_succ.groupby("JRNL_ID")["title"].count()
q1_jrnl_df.New_Additions.fillna(0, inplace=True)
q1_jrnl_df["New_Additions"] = q1_jrnl_df.New_Additions.astype(int)

with open("q1_jrnl_df","wb") as p:
        pickle.dump(q1_jrnl_df, p)

with open("q1_jrnl_df_copy","wb") as p:
        pickle.dump(q1_jrnl_df, p)

In [44]:
# Adding new articles to date_df & saving:

q1_date_df = pd.concat([q1_date_df,k5k_new_succ],verify_integrity=True)

with open("q1_date_df","wb") as p:
        pickle.dump(q1_date_df, p)

with open("q1_date_df_copy","wb") as p:
        pickle.dump(q1_date_df, p)

In [47]:
# Adding new articles to remainder_df & saving:

with open("q1_remainder_df", "rb") as fp:
    q1_remainder_df = pickle.load(fp)

q1_remainder_df = pd.concat([q1_remainder_df, k5k_new_rem_df.set_index("DOI")], verify_integrity=True)

with open("q1_remainder_df","wb") as p:
        pickle.dump(q1_remainder_df, p)

In [53]:
# Saving datasets to the run folder:

with open("7e_variables/last_5k_run_Sum/k5k_new_df","wb") as p:
        pickle.dump(k5k_new_df, p)

with open("7e_variables/last_5k_run_Sum/k5k_new_succ","wb") as p:
        pickle.dump(k5k_new_succ, p)

with open("7e_variables/last_5k_run_Sum/k5k_new_rem_df","wb") as p:
        pickle.dump(k5k_new_rem_df, p)

In [ ]:
# Deleting unnec. variables:

del q1_remainder_df
del k5k_new_rem_df
del k5k_new_df
del k5k_new_succ

In [55]:
with open("7b_variables/q1_a_ss", "rb") as fp:
    q1_a_ss = pickle.load(fp)

In [56]:
after_k5k_add_pivot = pd.pivot_table(data=q1_date_df, columns="JRNL_YEAR", index="Chosen_SA", values="title", aggfunc="count")

In [93]:
(q1_a_ss - after_k5k_add_pivot).applymap(lambda x: x if x>0 else 0)

Year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Sbj_Area,,,,,,,,,,,
Agricultural and Biological Sciences,0,0,0,0,0,0,0,0,0,0,0
Arts and Humanities,0,0,0,0,0,0,0,0,0,0,0
"Biochemistry, Genetics and Molecular Biology",0,0,0,0,0,0,0,0,0,0,0
"Business, Management and Accounting",0,0,0,0,0,0,0,0,0,0,0
Chemical Engineering,0,0,0,0,0,0,0,0,0,0,0
Chemistry,0,0,0,0,0,0,0,0,0,0,0
Computer Science,0,0,0,0,0,0,0,0,0,0,0
Decision Sciences,0,0,0,0,0,0,0,0,0,0,0
Dentistry,2,8,0,26,53,24,5,38,0,0,2


In [81]:
q1_a_ss.sum().sum()

171936

----

### Quick Analysis of q1_date_df:

In [83]:
q1_date_df.columns

Index(['JRNL_ID', 'ISSN', 'issued', 'container-title', 'publisher', 'title',
       'author', 'subject', 'references-count', 'is-referenced-by-count',
       'published', 'published-print', 'published-online', 'direct_url',
       'Dates', 'Keywords', 'Chosen_SA', 'JRNL_YEAR'],
      dtype='object')

In [84]:
q1_date_df["publ_year"] = q1_date_df.published.map(lambda x: x["date-parts"][0][0])

In [86]:
q1_date_df[q1_date_df.publ_year != q1_date_df.JRNL_YEAR]

,JRNL_ID,ISSN,issued,container-title,publisher,title,author,subject,references-count,is-referenced-by-count,published,published-print,published-online,direct_url,Dates,Keywords,Chosen_SA,JRNL_YEAR,publ_year
DOI,,,,,,,,,,,,,,,,,,,
10.1525/mp.2014.31.4.372,1_15782,"(0730-7829, 1533-8312)","{'date-parts': [[2012, 12]]}",[Music Perception],University of California Press,[Effects of Metrical Encoding on Melody Recogn...,"[{'given': 'Stefanie', 'family': 'Acevedo', 's...",[Music],45.0,3,"{'date-parts': [[2014, 4, 1]]}","{'date-parts': [[2014, 4, 1]]}",NaN,https://sci.bban.top/pdf/10.1525/mp.2014.31.4....,"[October 6, 2012, September 17, 2013]","[(melodies, 0.4663), (melodic, 0.4181), (audit...",Arts and Humanities,2012,2014
10.1525/mp.2012.29.3.285,1_15776,"[0730-7829, 1533-8312]","{'date-parts': [[2011, 12]]}",[Music Perception],University of California Press,[Intersecting Factors Lead to Absolute Pitch A...,"[{'given': 'Sarah J.', 'family': 'Wilson', 'se...",[Music],9.0,23,"{'date-parts': [[2012, 2, 1]]}","{'date-parts': [[2012, 2, 1]]}",NaN,https://sci.bban.top/pdf/10.1525/mp.2012.29.3....,"[August 23, 2010, June 22, 2011]","[(pitch, 0.4806), (tones, 0.4063), (pitches, 0...",Arts and Humanities,2011,2012
10.1525/mp.2012.29.3.255,1_15776,"[0730-7829, 1533-8312]","{'date-parts': [[2011, 12]]}",[Music Perception],University of California Press,[Working Memory for Tonal and Atonal Sequences...,"[{'given': 'Katrin', 'family': 'Schulze', 'seq...",[Music],0.0,30,"{'date-parts': [[2012, 2, 1]]}","{'date-parts': [[2012, 2, 1]]}",NaN,https://sci.bban.top/pdf/10.1525/mp.2012.29.3....,"[August 3, 2010, June 10, 2011]","[(tones, 0.4389), (auditory, 0.4237), (nonmusi...",Arts and Humanities,2011,2012


In [98]:
q1_date_df["Submitted_Date"] = q1_date_df.Dates.map(lambda x: x[0])

In [100]:
q1_date_df["Accepted_Date"] = q1_date_df.Dates.map(lambda x: x[1])

In [109]:
import regex as re

In [117]:
len(q1_date_df[q1_date_df.Submitted_Date.map(lambda x: bool(re.search("[A-Za-z]",x)))])

186352

In [119]:
len(q1_date_df[~q1_date_df.Submitted_Date.map(lambda x: bool(re.match("[A-Za-z]",x)))])

146025

In [121]:
len(q1_date_df[~q1_date_df.Submitted_Date.map(lambda x: bool(re.search("[A-Za-z]",x)))])

150

In [120]:
def simpl_dt_converter(sub_d):
    try:
        return pd.to_datetime(sub_d, infer_datetime_format=True)
    except:
        return np.nan

In [123]:
q1_date_df["Submitted_datetime"] = q1_date_df.Submitted_Date.map(simpl_dt_converter)

In [126]:
q1_date_df["Published_Date"] = q1_date_df.published.map(lambda x: x["date-parts"][0])

In [136]:
q1_date_df["Published_datetime"] = q1_date_df.Published_Date.map(lambda x: "-".join([str(it) for it in x])).map(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

In [138]:
q1_date_df[q1_date_df.Submitted_datetime > q1_date_df.Published_datetime]

,JRNL_ID,ISSN,issued,container-title,publisher,title,author,subject,references-count,is-referenced-by-count,...,Dates,Keywords,Chosen_SA,JRNL_YEAR,publ_year,Submitted_Date,Accepted_Date,Submitted_datetime,Published_Date,Published_datetime
DOI,,,,,,,,,,,,,,,,,,,,,
10.1007/s10469-020-09598-5,1_232,"[0002-5232, 1573-8302]","{'date-parts': [[2020, 7]]}",[Algebra and Logic],Springer Science and Business Media LLC,[Polynomially Computable Structures with Finit...,"[{'given': 'P. E.', 'family': 'Alaev', 'sequen...","[Logic, Analysis]",8.0,1,...,"[July 24, 2020, October 21, 2020]","[(computable, 0.4626), (substructures, 0.3979)...",Mathematics,2020,2020,"July 24, 2020","October 21, 2020",2020-07-24,"[2020, 7]",2020-07-01
10.2319/021320-103.1,1_576,[0003-3219],"{'date-parts': [[2020, 9, 14]]}",[The Angle Orthodontist],The Angle Orthodontist (EH Angle Education & R...,[Local administration of high-dose diabetes me...,"[{'given': 'Wei-Ren', 'family': 'Shen', 'seque...",[Orthodontics],33.0,3,...,"[February 2021, June 2020]","[(orthodontics, 0.3831), (molars, 0.3552), (os...",Dentistry,2020,2020,February 2021,June 2020,2021-02-01,"[2020, 9, 14]",2020-09-14
10.1115/1.4029146,1_671,"[0003-6900, 2379-0407]","{'date-parts': [[2018, 1, 1]]}",[Applied Mechanics Reviews],ASME International,[Discussion of “A Review of Thickness-Accommod...,"[{'given': 'Daniel A.', 'family': 'McAdams', '...",[Mechanical Engineering],25.0,1,...,"[March 14, 2018, None]","[(folding, 0.5963), (foldability, 0.5825), (fo...",Engineering,2018,2018,"March 14, 2018",None,2018-03-14,"[2018, 1, 1]",2018-01-01
10.1080/00049158.2010.10676313,1_878,[0004-9158],"{'date-parts': [[2010, 1]]}",[Australian Forestry],Informa UK Limited,[Insect defoliation of mixed-species eucalypts...,"[{'given': 'N. G.', 'family': 'Collett', 'sequ...",[Forestry],29.0,3,...,"[18 February 2010, None]","[(insectrelated, 0.5514), (pests, 0.4953), (pe...",Agricultural and Biological Sciences,2010,2010,18 February 2010,None,2010-02-18,"[2010, 1]",2010-01-01
10.1163/1568539x-00003463,1_942,"[0005-7959, 1568-539X]",{'date-parts': [[2017]]},[Behaviour],Brill,[The role of abiotic and biotic cues in burrow...,"[{'given': 'Mallary', 'family': 'Clay', 'seque...","[Behavioral Neuroscience, Animal Science and Z...",82.0,5,...,"[9 February 2017, 15 November 2017]","[(foraging, 0.4524), (aquatic, 0.4192), (habit...",Agricultural and Biological Sciences,2017,2017,9 February 2017,15 November 2017,2017-02-09,[2017],2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10.1177/2331216516673833,1_65764,"[2331-2165, 2331-2165]","{'date-parts': [[2016, 1]]}",[Trends in Hearing],SAGE Publications,[Social Context and Hearing Aid Adoption],"[{'given': 'Gurjit', 'family': 'Singh', 'seque...","[Speech and Hearing, Otorhinolaryngology]",41.0,12,...,"[27 June 2016, 21 September 2016]","[(hearing, 0.4609), (hearingimpaired, 0.4431),...",Health Professions,2016,2016,27 June 2016,21 September 2016,2016-06-27,"[2016, 1]",2016-01-01
10.3415/vcot-13-01-0018,1_66742,"[0932-0814, 2567-6911]",{'date-parts': [[2013]]},[Veterinary and Comparative Orthopaedics and T...,Georg Thieme Verlag KG,[Measurement of the patellar tendon-tibial pla...,"[{'given': 'C.', 'family': 'Bismuth', 'sequenc...","[General Veterinary, Animal Science and Zoology]",36.0,26,...,"[January 23, 2013, July 19, 2013]","[(ligament, 0.4392), (ligaments, 0.4352), (vet...",Veterinary,2013,2013,"January 23, 2013","July 19, 2013",2013-01-23,[2013],2013-01-01
10.3415/vcot-14-03-0041,1_66743,"[0932-0814, 2567-6911]",{'date-parts': [[2014]]},[Veterinary and Comparative Orthopaedics and T...,Georg Thieme Verlag KG,[Lameness associated with tarsal haemarthrosis...,"[{'given': 'E.', 'family': 'Mardell', 'sequenc...","[General Veterinary, Animal Science and Zoology]",28.0,4,...,"[March 5, 2014, August 26, 2014]","[(thrombocytopeniaat, 0.4112), (haemarthrosis,...",Veterinary,2014,2014,"March 5, 2014","August 26, 2014",2014-03-05,[2014],2014-

In [139]:
with open("q1_date_df_wip","wb") as p:
        pickle.dump(q1_date_df, p)